In [3]:
import numpy as np
import pandas as pd
import string
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


## Data

In [14]:
fake= pd.read_csv('data/Fake.csv')
true= pd.read_csv('data/True.csv')


In [15]:
fake.head(5)


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [16]:
#Cleaning Data
true = true.drop(columns=['title', 'subject', 'date'])
fake = fake.drop(columns=['title', 'subject', 'date'])

#Adding 'isfake'
true['isfake']= 0
fake['isfake']= 1

#Combining df
data= pd.concat([true, fake], axis=0, ignore_index=True)
data


,text,isfake
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0
...,...,...
44893,21st Century Wire says As 21WIRE reported earl...,1
44894,21st Century Wire says It s a familiar theme. ...,1
44895,Patrick Henningsen 21st Century WireRemember ...,1
44896,21st Century Wire says Al Jazeera America will...,1


## Query Preprocessing

In [2]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [3]:
def cleaning(query):

    query = query.strip()
    query = query.lower()
    query = ''.join(char for char in query if not char.isdigit())

    for punctuation in string.punctuation:
        query = query.replace(punctuation, '')


    tokenized_query = word_tokenize(query)
    stop_words = set(stopwords.words('english'))

    tokenized_query_cleaned = [
        w for w in tokenized_query if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n")
        for word in tokenized_query_cleaned
    ]

    cleaned_query= ' '.join(word for word in lemmatized)

    return cleaned_query


## Pipeline Contruction

In [ ]:
pipeline= make_pipeline()
